### Experiment Summary
Finetune the last layer of a naturally trained model using adversarial loss.
We **do** re-initialize the last layer.

# Setup

### Import dependencies

In [1]:
import os
os.environ["NOTEBOOK_MODE"] = "1" # for robustness library

from robustness import model_utils, datasets, train, defaults
from robustness.datasets import CIFAR

# We use cox (http://github.com/MadryLab/cox) to log, store and analyze
# results. Read more at https//cox.readthedocs.io.
from cox.utils import Parameters
import cox.store

import torch
[torch.cuda.get_device_name(a) for a in range(torch.cuda.device_count())]

/ufs/ttw/quada/miniconda3/envs/robustness/lib/python3.6/site-packages/robustness/train.py:24: UserWarning: Could not import amp.
  warnings.warn('Could not import amp.')


['GeForce RTX 3090']

### Download models
From https://github.com/MadryLab/robustness.
Two models downloaded:
- CIFAR10 Linf-norm (ResNet50), ε = 0     (natural training)
- CIFAR10 Linf-norm (ResNet50), ε = 8/255

In [2]:
!mkdir -p data/models
!wget -q -O data/models/cifar_linf_8.pt "https://www.dropbox.com/s/c9qlt1lbdnu9tlo/cifar_linf_8.pt?dl=1"
!wget -q -O data/models/cifar_nat.pt    "https://www.dropbox.com/s/yhpp4yws7sgi6lj/cifar_nat.pt?dl=1"

### Load dataset and models

In [2]:
DS = CIFAR("data")
TRAIN_LOADER, VAL_LOADER = DS.make_loaders(
    batch_size=128, workers=12
)

==> Preparing dataset cifar..
Files already downloaded and verified
Files already downloaded and verified


In [3]:
M_NAT, _ = model_utils.make_and_restore_model(
    arch="resnet50",
    resume_path="data/models/cifar_nat.pt",
    dataset=DS, 
)
M_NAT.eval()

M_ADV, _ = model_utils.make_and_restore_model(
    arch="resnet50",
    resume_path="data/models/cifar_linf_8.pt",
    dataset=DS, 
)
M_ADV.eval();

=> loading checkpoint 'data/models/cifar_nat.pt'
=> loaded checkpoint 'data/models/cifar_nat.pt' (epoch 190)
=> loading checkpoint 'data/models/cifar_linf_8.pt'
=> loaded checkpoint 'data/models/cifar_linf_8.pt' (epoch 153)


In [4]:
torch.manual_seed(42)

m_finetune, _ = model_utils.make_and_restore_model(
    arch="resnet50",
    resume_path="data/models/cifar_nat.pt",
    dataset=DS, 
)

for p1, p2 in zip(m_finetune.model.linear.parameters(), M_NAT.model.linear.parameters()):
    assert torch.equal(p1, p2)

m_finetune.model.linear.reset_parameters()
    
for p1, p2 in zip(m_finetune.model.linear.parameters(), M_NAT.model.linear.parameters()):
    assert not torch.equal(p1, p2)
    
m_finetune_params = m_finetune.model.linear.parameters()

=> loading checkpoint 'data/models/cifar_nat.pt'
=> loaded checkpoint 'data/models/cifar_nat.pt' (epoch 190)


### Set up Cox logging

In [5]:
COX_STORE = cox.store.Store("cox")

Logging in: /rfs/quada/ttw/code/adversarial-ntks/notebooks/discard-hypothesis/cox/114ceb62-9909-436a-8dfb-396ebc7cdde8


# Baseline accuracies

In [7]:
EVAL_ARGS = defaults.check_and_fill_args(
    args=Parameters({
        "adv_eval": 1,
        "out_dir": "eval_out",
        "constraint": 'inf', # L-inf PGD
        "eps": 8.0 / 255.0, # Epsilon constraint (L-inf norm)
        "attack_lr": 2.5 * 8 / 255 / 20,
        "attack_steps": 20
    }),
    arg_list=defaults.PGD_ARGS,
    ds_class=CIFAR
)

In [8]:
m_finetune.eval()
train.eval_model(
    model=m_finetune,
    args=EVAL_ARGS,
    loader=VAL_LOADER,
    store=COX_STORE
)

  0%|          | 0/79 [00:00<?, ?it/s]

  0%|          | 0/79 [00:00<?, ?it/s]

{'epoch': 0,
 'nat_prec1': tensor(5.8600, device='cuda:0'),
 'adv_prec1': tensor(0., device='cuda:0'),
 'nat_loss': 2.294361653518677,
 'adv_loss': 2.577602601623535,
 'train_prec1': nan,
 'train_loss': nan,
 'time': 218.8739378452301}

In [9]:
train.eval_model(
    model=M_NAT,
    args=EVAL_ARGS,
    loader=VAL_LOADER,
    store=COX_STORE
)

  0%|          | 0/79 [00:00<?, ?it/s]

  0%|          | 0/79 [00:00<?, ?it/s]

{'epoch': 0,
 'nat_prec1': tensor(95.2500, device='cuda:0'),
 'adv_prec1': tensor(0., device='cuda:0'),
 'nat_loss': 0.19557504131793976,
 'adv_loss': 26.358837536621095,
 'train_prec1': nan,
 'train_loss': nan,
 'time': 207.5380458831787}

# Adversarial finetuning 

In [6]:
TRAIN_ARGS = Parameters({
    "out_dir": "train_out",
    "adv_train": 1, # Use adversarial training
    "constraint": 'inf', # L-inf PGD
    "eps": 8.0 / 255.0, # Epsilon constraint (L-inf norm)
    "attack_lr": 2.5 * 8 / 255 / 20,
    "attack_steps": 20
})

# Fill whatever parameters are missing from the defaults
TRAIN_ARGS = defaults.check_and_fill_args(
    TRAIN_ARGS, defaults.TRAINING_ARGS, CIFAR
)
TRAIN_ARGS = defaults.check_and_fill_args(
    TRAIN_ARGS, defaults.PGD_ARGS, CIFAR
)

In [ ]:
# Train a model
m_finetune.train()
train.train_model(
    model=m_finetune,
    update_params=m_finetune_params,
    args=TRAIN_ARGS,
    loaders=(TRAIN_LOADER, VAL_LOADER),
    store=COX_STORE
)

  0%|          | 0/391 [00:00<?, ?it/s]

  0%|          | 0/79 [00:00<?, ?it/s]

  0%|          | 0/79 [00:00<?, ?it/s]

  0%|          | 0/391 [00:00<?, ?it/s]

  0%|          | 0/391 [00:00<?, ?it/s]